# Analysing Data - Assignment 3 

Use Python’s Ollama library to prompt a model to annotate/classify song lyrics with their associated genre. Use the data (genreLyrics.csv) that is attached below.

Implement 1) Zero-shot prompting and 2) Few-Shot prompting.

Calculate the performance of both strategies of the model using Precision, Recall and F1 Score. Does performance vary by genre?

Hint: Investigate the output of the model before you calculate P-R-F1.

Provide a small discussion (½-1 page).
Which prompting strategy worked better, and why?

Did the model struggle with specific genres?

How could performance be improved?

Commit your notebook/code to GitHub. Do not forget to include the link to your repository when you upload your assignment. Make sure your code runs and is sufficiently documented (GitHub README and comments).


## Downloading Ollama

In [1]:
pip install ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 642.6 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.6/442.6 kB 3.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 3.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.3 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.18.4
    Uninstalling pydantic_core-2.18.4:
      Successfully uninstalled pydantic_core-2.18.4
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.7.4
    Uninstalling pydantic-2.7.4:
      Successfully uninstalled pydantic-2.7.4
Note: you may need to restart the kernel to use updated packages.


In [52]:
import ollama

#testing
response = ollama.chat(model='llama2', messages=[
    {
        'role': 'user',
        'content': 'Why is the sky blue?',
    },
])
print(response['message']['content'])


The sky appears blue because of a phenomenon called Rayleigh scattering, which occurs when sunlight passes through the Earth's atmosphere. The atmosphere scatters the shorter, blue wavelengths of light more than the longer, red wavelengths, causing the blue light to be scattered in all directions and reach our eyes from everywhere in the sky. This is known as the "blue sky effect."

The reason for this scattering is that the molecules of gases in the atmosphere, such as nitrogen and oxygen, are much larger than the wavelength of light. As a result, they act like very small mirrors, reflecting the light in all directions. The blue light is scattered more than other colors because it has a shorter wavelength, which means that it is more easily reflected by the molecules in the atmosphere.

In addition to Rayleigh scattering, the sky can also appear blue due to the scattering of light by particles in the atmosphere, such as dust and water droplets. This type of scattering is known as Mie

## Read dataset, pre-process and make sure column names are as expected

In [68]:
import pandas as pd

url = "https://raw.githubusercontent.com/natalievxr/AD_Assignment3/main/genreLyrics.csv"
df = pd.read_csv(url, sep="\t", on_bad_lines="skip")  # it had trouble reading the file as the commas in the lyrics were confusing
df.columns = df.columns.str.strip()  # remove unwanted spaces from column names
print(df.columns)  # Verify the column names



Index(['Unnamed: 0', 'genre', 'lyrics'], dtype='object')


In [98]:
# use first 5 rows of the dataset
df_small = df.head(5) 

# define the zero-shot function
def zero_shot_classify(lyrics):
    prompt = f"Classify the following lyrics into a genre: {lyrics}"
    response = ollama.generate(model='llama2', prompt=prompt)  # specify the model
    print(response)  # print the entire response to inspect its structure
    return response  # temporarily return the whole response for inspection

# apply the zero-shot function to the smaller dataset
df_small['predicted_genre_zero_shot'] = df_small['lyrics'].apply(zero_shot_classify)

# print the results
print(df_small[['lyrics', 'predicted_genre_zero_shot']])

model='llama2' created_at='2025-03-29T20:15:11.428014Z' done=True done_reason='stop' total_duration=135683250106 load_duration=87431365 prompt_eval_count=275 prompt_eval_duration=35110443257 eval_count=311 eval_duration=100479585655 response="Based on the lyrics provided, I would classify this song as being in the Alternative Rock genre. The lyrics have a introspective and melancholic tone, with themes of self-discovery, change, and reflection. The use of repetition and imagery also adds to the emotional impact of the lyrics.\n\nSome key elements that suggest this song belongs to the Alternative Rock genre are:\n\n1. Lyrical themes: The lyrics touch on themes that are common in Alternative Rock, such as self-discovery, change, and reflection. These themes are often explored through introspective and emotional lyrics, which is a hallmark of the genre.\n2. Musical structure: The song's musical structure, with its use of repetition and build-ups, also aligns with the conventions of Altern

/var/folders/h1/m5x6sfs57nlcs3l70snmm87c0000gn/T/ipykernel_34001/3302710784.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small['predicted_genre_zero_shot'] = df_small['lyrics'].apply(zero_shot_classify)


                                              lyrics  \
0  Hey, if you were right I'd chase away\nAll the...   
1  There's something about the way we fit\nThere'...   
2  One drop in the ocean\nCould be that magic pot...   
3  I'm so tired of being here\nSuppressed by all ...   
4  Yeah, what, Vast Aire,; Shell Shock..\nIt's th...   

                           predicted_genre_zero_shot  
0  model='llama2' created_at='2025-03-29T20:15:11...  
1  model='llama2' created_at='2025-03-29T20:17:05...  
2  model='llama2' created_at='2025-03-29T20:17:50...  
3  model='llama2' created_at='2025-03-29T20:19:13...  
4  model='llama2' created_at='2025-03-29T20:23:16...  


In [100]:
# define the few-shot function
few_shot_examples = [
    "Baby, I'm just gonna shake, shake, shake, shake, shake\nI shake it off → Pop",  
    "Cause I hate that stupid old pickup truck you never let me drive → Country",  
    "Now you hang from my lips\nLike the Gardens of Babylon\nWith your boots beneath my bed\nForever is the sweetest con → Indie Folk"
]

def few_shot_classify(lyrics):
    examples = "\n".join(few_shot_examples)
    prompt = f"Classify the following lyrics into a genre based on these examples:\n{examples}\n\nLyrics: {lyrics}"
    response = ollama.generate(model='llama2', prompt=prompt)  # specify the model
    print(response)  # print the entire response to inspect its structure
    return response  # temporarily return the whole response for inspection

# apply the few-shot function to the smaller dataset
df_small['predicted_genre_few_shot'] = df_small['lyrics'].apply(few_shot_classify)

# print the results
print(df_small[['lyrics', 'predicted_genre_few_shot']])

model='llama2' created_at='2025-03-29T20:34:02.079397Z' done=True done_reason='stop' total_duration=156337621801 load_duration=11427198732 prompt_eval_count=374 prompt_eval_duration=47374387002 eval_count=419 eval_duration=97466620306 response='Based on the provided lyrics, here are my classifications:\n\n* "Hey, if you were right I\'d chase away / All the reasons for my old desire to change" - Indie Folk\n\t+ This verse has a introspective and emotional tone, with a focus on personal growth and change. The language is simple and straightforward, but also poetic and evocative.\n* "But the right words, don\'t improvise the ideals / My body sails into a passage waiting in vain" - Alternative/Indie Rock\n\t+ This verse has a more upbeat and energetic tone, with a focus on language play and imagery. The use of the word "sails" and the mention of a "passage" suggest a sense of movement and progression.\n* "It might just be so uninviting, is it comical / From until now I\'ve gone to somethin

/var/folders/h1/m5x6sfs57nlcs3l70snmm87c0000gn/T/ipykernel_34001/2223633238.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small['predicted_genre_few_shot'] = df_small['lyrics'].apply(few_shot_classify)


                                              lyrics  \
0  Hey, if you were right I'd chase away\nAll the...   
1  There's something about the way we fit\nThere'...   
2  One drop in the ocean\nCould be that magic pot...   
3  I'm so tired of being here\nSuppressed by all ...   
4  Yeah, what, Vast Aire,; Shell Shock..\nIt's th...   

                            predicted_genre_few_shot  
0  model='llama2' created_at='2025-03-29T20:34:02...  
1  model='llama2' created_at='2025-03-29T20:37:37...  
2  model='llama2' created_at='2025-03-29T20:38:47...  
3  model='llama2' created_at='2025-03-29T20:40:18...  
4  model='llama2' created_at='2025-03-29T20:43:44...  


## Calculating Precision, F1 score and Recall of both strategies

In [118]:
from sklearn.metrics import precision_score, recall_score, f1_score

# data from the table
data = {
    "Song": [1, 2, 3, 4, 5],
    "Zero Shot": ["Alternative Rock", "Punk Rock", "Pop", "Emo/sad core", "Experimental hip hop"],
    "Few Shot": ["Indie Rock", "Hip hop/Rock", "Pop/rock", "Indie Folk", "Hip hop/Indie Folk"],
    "Correct Genre": ["Rock", "Rock", "Electronic", "Rock", "Hip hop"]
}

# convert to df
df = pd.DataFrame(data)

# convert genres to numerical labels
label_mapping = {"Rock": 0, "Electronic": 1, "Hip hop": 2, "Pop": 3}
correct_labels = df["Correct Genre"].replace(label_mapping)

# convert zero-shot and few-shot outputs to numerical labels
zero_shot_mapping = {"Alternative Rock": 0, "Punk Rock": 0, "Pop": 3, "Emo/sad core": 0, "Experimental hip hop": 2}
few_shot_mapping = {"Indie Rock": 0, "Hip hop/Rock": 2, "Pop/rock": 3, "Indie Folk": 0, "Hip hop/Indie Folk": 2}

zero_shot_labels = df["Zero Shot"].replace(zero_shot_mapping)
few_shot_labels = df["Few Shot"].replace(few_shot_mapping)

# compute Precision, Recall, and F1 Score for Zero-Shot 
zero_shot_precision = precision_score(correct_labels, zero_shot_labels, average="macro")
zero_shot_recall = recall_score(correct_labels, zero_shot_labels, average="macro")
zero_shot_f1 = f1_score(correct_labels, zero_shot_labels, average="macro")

# compute Precision, Recall, and F1 Score for Few-Shot 
few_shot_precision = precision_score(correct_labels, few_shot_labels, average="macro")
few_shot_recall = recall_score(correct_labels, few_shot_labels, average="macro")
few_shot_f1 = f1_score(correct_labels, few_shot_labels, average="macro")

# print result
print("Zero Shot Strategy:")
print(f"  Precision: {zero_shot_precision:.2f}")
print(f"  Recall: {zero_shot_recall:.2f}")
print(f"  F1 Score: {zero_shot_f1:.2f}\n")

print("Few Shot Strategy:")
print(f"  Precision: {few_shot_precision:.2f}")
print(f"  Recall: {few_shot_recall:.2f}")
print(f"  F1 Score: {few_shot_f1:.2f}")

Zero Shot Strategy:
  Precision: 0.50
  Recall: 0.50
  F1 Score: 0.50

Few Shot Strategy:
  Precision: 0.38
  Recall: 0.42
  F1 Score: 0.37


/var/folders/h1/m5x6sfs57nlcs3l70snmm87c0000gn/T/ipykernel_34001/2487867479.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  correct_labels = df["Correct Genre"].replace(label_mapping)
/var/folders/h1/m5x6sfs57nlcs3l70snmm87c0000gn/T/ipykernel_34001/2487867479.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  zero_shot_labels = df["Zero Shot"].replace(zero_shot_mapping)
/var/folders/h1/m5x6sfs57nlcs3l70snmm87c0000gn/T/ipykernel_34001/2487867479.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will b